In [1]:
import json
import pandas as pd 
import csv
from urlparse import urlparse
import numpy as np

In [2]:
%cd ~/Projects/creativemornings/Data

/Users/divyamehra/Projects/creativemornings/Data


In [3]:
with open('profiles.json', 'r') as f:
     data = json.load(f)

In [4]:
rawprofiledf = pd.DataFrame(data)

In [5]:
rawprofiledf = rawprofiledf.drop('activity', axis=1)
rawprofiledf = rawprofiledf.drop('answers', axis=1)
rawprofiledf = rawprofiledf.drop('qa', axis=1)
rawprofiledf = rawprofiledf.drop('skills', axis=1)
#rawprofiledf = rawprofiledf.drop('bio', axis=1)

In [6]:
rawprofiledf.head(5)

,attended,bio,city,desc,forhire,links,name,picture,questions,social,uploads,watchlater
0,0,"[<div class=""user-bio editable-wrap"">\n<div cl...",\narh\n,"[\nBuy Duratia 30mg at alldaygeneric.\n, \n]",[],[https://www.alldaygeneric.com/product/duratia...,"[\nMadeline Neves\n, \n]",[https://media.creativemornings.com/uploads/us...,[],"[https://twitter.com/alldaygeneric, https://fa...",[1],0
1,1,[],\narh\n,[],[],[],[\nDorte Holst\n],[/assets/avatars/cm-avatar-1.svg],[],[],[],0
2,1,[],\narh\n,[],[],[],[\nFlorence Lam\n],[/assets/avatars/cm-avatar-3.svg],[],[],[],0
3,0,[],\narh\n,[],[],[],[\nMartin Ludvigsen\n],[/assets/avatars/cm-avatar-1.svg],[],[],[],0
4,1,[],\narh\n,[],[],[],[\nMark Christensen\n],[/assets/avatars/cm-avatar-2.svg],[],[],[],0


In [7]:
rawprofiledf.shape # (224303, 12)

(224303, 12)

In [8]:
rawprofiledf.isnull().sum() # no null values

attended      0
bio           0
city          0
desc          0
forhire       0
links         0
name          0
picture       0
questions     0
social        0
uploads       0
watchlater    0
dtype: int64

In [9]:
rawprofiledf['city'] = rawprofiledf['city'].map(lambda x: x.strip('\n'))
rawprofiledf['name'] = rawprofiledf['name'].map(lambda x: x[0].strip('\n'))

In [10]:
rawprofiledf['attended'] = rawprofiledf['attended'].map(lambda x: int(x)) 
rawprofiledf['watchlater'] = rawprofiledf['watchlater'].map(lambda x: int(x))

In [11]:
rawprofiledf['picture'] = rawprofiledf['picture'].map(lambda x: str(x))

In [170]:
rawprofiledf['haspicture'] = np.where(rawprofiledf['picture'].str.contains("user", case=False, na=False), True, False)

In [171]:
rawprofiledf['haspicture'].value_counts()

False    135109
True      89194
Name: haspicture, dtype: int64

In [14]:
rawprofiledf.columns

Index([  u'attended',        u'bio',       u'city',       u'desc',
          u'forhire',      u'links',       u'name',    u'picture',
        u'questions',     u'social',    u'uploads', u'watchlater',
       u'haspicture'],
      dtype='object')

In [15]:
# ATTENDED EVENTS
rawprofiledf[rawprofiledf['attended'] > 5].describe() # 9.7% have attended more than 5 events (21,725) - with an average of 10
rawprofiledf[rawprofiledf['attended'] > 10].describe() # 3.5% have attended more than 10 events (7,516) - with an average of 16
rawprofiledf[rawprofiledf['attended'] == 0].describe() # 25% have never attended an event (56,435)
rawprofiledf['attended'].describe() # (25-50-75) 0 - 1 - 2; skewed right with a mean of 2.2

count    224303.000000
mean          2.168821
std           3.543649
min           0.000000
25%           0.000000
50%           1.000000
75%           2.000000
max          79.000000
Name: attended, dtype: float64

In [16]:
# CITY
rawprofiledf['city'].value_counts() # top cities are: nyc, atl, sf, la, sd, dc, mtl, chi, pdx, mow, van, & atx (>4K)
rawprofiledf[rawprofiledf['city'] == 'nyc']['attended'].describe() # 1.5 mean
rawprofiledf[rawprofiledf['city'] == 'mtl']['attended'].describe() # 2.4 mean
rawprofiledf[rawprofiledf['city'] == 'pdx']['attended'].describe() # 2.7 mean

count    4393.000000
mean        2.706123
std         3.953844
min         0.000000
25%         1.000000
50%         1.000000
75%         3.000000
max        38.000000
Name: attended, dtype: float64

In [17]:
# PICTURE
rawprofiledf['haspicture'].value_counts()
135109/224303 # 60% do not have a photo uploaded 

0

In [28]:
rawprofiledf['forhire'] = rawprofiledf['forhire'].map(lambda x: str(x))
rawprofiledf['bio'] = rawprofiledf['bio'].map(lambda x: str(x))
rawprofiledf['desc'] = rawprofiledf['desc'].map(lambda x: str(x))

In [34]:
rawprofiledf['forhire'].value_counts()
38087/224303 #17% are for hire

rawprofiledf[rawprofiledf['desc'] == '[]'].describe() # 111,549
111549/224303 #50% have no description

1698

In [162]:
rawprofiledf['jobseeker'] = np.where(rawprofiledf['forhire'].str.contains("hire", case=False, na=False), True, False)
rawprofiledf['jobseeker'].value_counts()

rawprofiledf['hasdesc'] = np.where(rawprofiledf['desc'] == '[]', False, True)
rawprofiledf['hasdesc'].value_counts()

rawprofiledf['hasbio'] = np.where(rawprofiledf['bio'] == '[]', False, True)
rawprofiledf['hasbio'].value_counts()

rawprofiledf['uploadcount'] = [len(x) for x in rawprofiledf['uploads']]
rawprofiledf['uploadcount'].value_counts()

rawprofiledf['qcount'] = [len(x) for x in rawprofiledf['questions']]
rawprofiledf['qcount'].value_counts()

0    203590
3      3741
1      3284
8      2810
9      2514
2      2339
7      1590
4      1507
5      1477
6      1451
Name: qcount, dtype: int64

In [103]:
rawprofiledf['strlinks'] = rawprofiledf['links'].map(lambda x: str(x))

In [156]:
rawprofiledf['haslinks'] = np.where(rawprofiledf['strlinks'] == '[]', False, True)
rawprofiledf['haslinks'].value_counts()

False    158266
True      66037
Name: haslinks, dtype: int64

In [157]:
rawprofiledf['twitter'] = np.where(rawprofiledf['strlinks'].str.contains("twitter", case=False, na=False), True, False)
rawprofiledf['twitter'].value_counts()

False    183352
True      40951
Name: twitter, dtype: int64

In [158]:
rawprofiledf['tco'] = np.where(rawprofiledf['strlinks'].str.contains("t.co", case=False, na=False), True, False)
rawprofiledf['tco'].value_counts()

False    212585
True      11718
Name: tco, dtype: int64

In [160]:
rawprofiledf['facebook'] = np.where(rawprofiledf['strlinks'].str.contains("facebook", case=False, na=False), True, False)
rawprofiledf['facebook'].value_counts()

False    198679
True      25624
Name: facebook, dtype: int64

In [159]:
rawprofiledf['linkedin'] = np.where(rawprofiledf['strlinks'].str.contains("linkedin", case=False, na=False), True, False)
rawprofiledf['linkedin'].value_counts()

False    221119
True       3184
Name: linkedin, dtype: int64

In [161]:
rawprofiledf['instagram'] = np.where(rawprofiledf['strlinks'].str.contains("instagram", case=False, na=False), True, False)
rawprofiledf['instagram'].value_counts()

False    219479
True       4824
Name: instagram, dtype: int64

In [163]:
rawprofiledf.columns 

Index([   u'attended',         u'bio',        u'city',        u'desc',
           u'forhire',       u'links',        u'name',     u'picture',
         u'questions',      u'social',     u'uploads',  u'watchlater',
        u'haspicture',   u'jobseeker',     u'hasdesc',      u'hasbio',
       u'uploadcount',      u'qcount',    u'strlinks',     u'twitter',
          u'haslinks',    u'facebook',         u'tco',    u'linkedin',
         u'instagram'],
      dtype='object')

In [172]:
binarydf = rawprofiledf.filter(['name','city','attended', 'jobseeker', 'hasdesc', 'haspicture', 'hasbio',
                      'qcount', 'uploadcount', 'haslinks', 'twitter', 'tco', 'facebook', 'linkedin', 'instagram',
                      'watchlater'], axis=1)

In [173]:
binarydf.columns

Index([u'name', u'city', u'attended', u'jobseeker', u'hasdesc', u'haspicture',
       u'hasbio', u'qcount', u'uploadcount', u'haslinks', u'twitter', u'tco',
       u'facebook', u'linkedin', u'instagram', u'watchlater'],
      dtype='object')

In [174]:
binarydf.describe()

,attended,qcount,uploadcount,watchlater
count,224303.000000,224303.000000,224303.000000,224303.000000
mean,2.168821,0.434858,0.148852,0.023045
std,3.543649,1.606031,0.696207,0.705380
min,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,0.000000,0.000000
max,79.000000,9.000000,4.000000,153.000000


In [175]:
binarydf.jobseeker = binarydf.jobseeker.astype(int)
binarydf.hasdesc = binarydf.hasdesc.astype(int)
binarydf.hasbio = binarydf.hasbio.astype(int)
binarydf.haspicture = binarydf.haspicture.astype(int)
binarydf.haslinks = binarydf.haslinks.astype(int)
binarydf.twitter = binarydf.twitter.astype(int)
binarydf.tco = binarydf.tco.astype(int)
binarydf.linkedin = binarydf.linkedin.astype(int)
binarydf.facebook = binarydf.facebook.astype(int)
binarydf.instagram = binarydf.instagram.astype(int)

In [178]:
binarydf.describe()

,attended,jobseeker,hasdesc,haspicture,hasbio,qcount,uploadcount,haslinks,twitter,tco,facebook,linkedin,instagram,watchlater
count,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000,224303.000000
mean,2.168821,0.169802,0.502686,0.397650,0.192686,0.434858,0.148852,0.294410,0.182570,0.052242,0.114238,0.014195,0.021507,0.023045
std,3.543649,0.375459,0.499994,0.489413,0.394409,1.606031,0.696207,0.455778,0.386314,0.222515,0.318101,0.118295,0.145066,0.705380
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,2.000000,0.000000,1.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,79.000000,1.000000,1.000000,1.000000,1.000000,9.000000,4.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,153.000000


In [179]:
binarydf = binarydf.rename(columns={'hasdesc': 'description', 'haspicture': 'picture', 'hasbio': 'bio',
                                   'haslinks': 'links', 'uploadcount': 'uploads', 'qcount': 'questions'})

In [183]:
binarydf.shape

(224303, 16)

In [184]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df.to_csv(file_name, encoding='utf-8', index=False)
